# Create from Notebook Example
## Introduction
In this example, we show the absolute fastest way to get a function up and running in CDF. We do this by defining the function in the notebook itself, and when creating the function in CDF, we simply point to our function definition.

**Documentation**:
We will be using the Python SDK of Cognite Functions for which the documentation can be found here:

https://cognite-sdk-experimental.readthedocs-hosted.com/en/latest/cognite.html#functions

## Setup
We first import the experimental CogniteClient and instantiate it with an API-key, which we retrieve via `getpass`.

In [ ]:
from getpass import getpass
from cognite.experimental import CogniteClient

project = # Fill in your project here
base_url = # Fill in your base url here

api_key = getpass()
client = CogniteClient(
    api_key=api_key,
    project=project,
    client_name="DSHub",
    base_url=base_url
)

## Defining our function
Next we define the function we wish to deploy. In this example, we simply list the assets and return the name of one the assets.

In [ ]:
def handle(client, data):
    asset_no = data["assetNo"]
    assets = client.assets.list()
    print(assets[asset_no].name)
    return {
        "assetName": assets[asset_no].name
    }

The function must be named `handle`, and arguments to handle must be a subset of `(client, data, secrets, function_call_info)`. These arguments serve the following purpose:
* `client` is a pre-instantiated `CogniteClient` which is automatically available inside our function call-time, provided the function was created with an API-key, which we do below.
* `data` is any data you send to your function. In this case we will send in a dictionary on the form `data = {"assetNo": 1}`. Note that `data` must be json serializable.
* `secrets` is a dictionary of key/value pairs which serves as a way to send sensitive information to your function. We don't use secrets in this particular example.
* `function_call_info` is a dictionary of key/value pairs providing the `function_id` - and if the function is scheduled - the keys `schedule_id` and `scheduled_time` 

## Deploying the function to CDF
Next we deploy the function to CDF. We do this by setting the argument `function_handle` equal to `handle`, the function we defined above. Additionally, we give it a name, an external id and an API-key. The API-key is necessary, since we will get a `CogniteClient` inside our function, which will be instantiated with this key. If your function doesn't need a client, you don't need to provide an API-key when creating it.

In [ ]:
name = # Fill in a unique name here
external_id = name
function = client.functions.create(
    name=name, 
    external_id=external_id, 
    function_handle=handle, 
    api_key=api_key
)

The function will start out with status `Queued`, then `Deploying` and finally `Ready`. This typically takes about 3-5 mihutes. To get the latest function status, run `function.update()`. **NB**: All calls to the function before it's `Ready` will fail.

In [ ]:
# Repeat this cell until status is Ready
function.update()
function.status

The function status can also be monitored through the functions UI.

## Calling the function
We can call the function directly from the `function` object as follows

In [ ]:
call = function.call(data={"assetNo": 1})
call

The call object contains metadata about the call, but not the response itself. To retrieve the response, we do

In [ ]:
call.get_response()

Similarly, you can get the logs by

In [ ]:
call.get_logs()

All print statements will turn up in the logs, or if an exception is raised, the traceback will appear.

## Cleanup
Finally, we clean up after ourselves by deleting the function.

In [ ]:
client.functions.delete(id=function.id)